In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv', parse_dates=["epoch"])
test = pd.read_csv('test.csv', parse_dates=["epoch"])

In [ ]:
data = pd.concat([train, test], axis=0)

In [4]:
data['dy'] = data.epoch.dt.dayofyear
data['sec_year'] = (data.epoch.dt.dayofyear - 1)*24*60*60 + (data.epoch.dt.hour - 0)*60*60 + (data.epoch.dt.minute - 0)*60 + data.epoch.dt.second

In [5]:
FCOLS= [
    'sec_year',
    'Vx', 
    'Vy',
    'Vz',
    'x',
    'y',
    'z', 
    'id', 
    'dy'
]

In [ ]:
mas_all = []
v_cnt = 0
for v_sat_id in pd.unique(train.sat_id):   
    v_cnt = v_cnt + 1
    print(v_cnt)
    
    mas = data[data.sat_id == v_sat_id][FCOLS].values
    v_prev = -1
    mas_new = []
    j = -1
    z = 0
    zzz = 0
    for i in mas:
        if i[8] < 32:
            if i[0] != v_prev:
                mas_new.append(i)
                j = j + 1
        else:
            if zzz == 0:
                znam = int(j/24)
                zzz = 1
            
            j = j + 1
            mas_new.append(i)
            v_delta = [0, 0, 0, 0, 0, 0]
            if i[0] == v_prev:
                z = 24
                
            if z > 0:  
                zz = 0
                for q in range(49, 73):
                    if mas_new[j - q][0] == mas_new[j - q - 1][0]:
                        zz = zz + 1
                        
                for k in range(6):
                    v_delta[k] = (mas_new[j - 25][k + 1] - mas_new[j - 73 - zz][k + 1])/2.0

                for k in range(1, 7):
                    mas_new[j][k] = mas_new[j - 25][k] + v_delta[k - 1]
                z = z - 1
            else:
                zz = 0
                for q in range(24, 48):
                    if mas_new[j - q][0] == mas_new[j - q - 1][0]:
                        zz = zz + 1
                for q in range(48 + zz, 72 + zz):
                    if mas_new[j - q][0] == mas_new[j - q - 1][0]:
                        zz = zz + 1
                    
                for k in range(6):
                    v_delta[k] = (mas_new[j - 24][k + 1] - mas_new[j - 72 - zz][k + 1])/2.0

                for k in range(1, 7):
                    mas_new[j][k] = mas_new[j - 24][k] + v_delta[k - 1]    
                
            mas_all.append(mas_new[j])
            #print(str(mas_new[j][7]) + " - " + str(mas_new[j][4]) + " - " + str(v_delta[4]) + " - " + str(z))
        v_prev = i[0]

In [28]:
data_otvet=pd.DataFrame(mas_all, columns = ['sec_year', 'Vx', 'Vy', 'Vz', 'x', 'y', 'z', 'id', 'dy'])

In [29]:
data_otvet.id = data_otvet.id.astype(int32)

In [30]:
data_otvet[['id', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz']].to_csv('otvet_15.csv', index=False)